<a href="https://colab.research.google.com/github/hemanth-sunkireddy/kaggle-iiith-location-challenge/blob/main/model-v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install torch torchvision --force-reinstall

from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/drive/MyDrive/SMAI_Project/SMAI_Project/images_train /content/
!cp -r /content/drive/MyDrive/SMAI_Project/SMAI_Project/images_val /content/

In [ ]:
!cp -r /content/drive/MyDrive/SMAI_Project/SMAI_Project/labels_train.csv /content/
!cp -r /content/drive/MyDrive/SMAI_Project/SMAI_Project/labels_val.csv /content/

In [ ]:
train_csv_path = 'labels_train.csv'
val_csv_path = 'labels_val.csv'
train_img_dir = '/content/images_train'
val_img_dir = '/content/images_val'
lat_long_output_csv = '/content/lat-long.csv'
region_output_csv = '/content/region.csv'
angle_output_csv = '/content/angle.csv'

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"💻 Using device: {device}")

In [ ]:
train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
])


val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Region Prediction

In [ ]:
class RegionDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        print(f"📦 Initialized dataset with {len(self.df)} samples from {img_dir}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')
        label = row['Region_ID'] - 1  # 0-indexed

        if self.transform:
            image = self.transform(image)

        return image, label

train_dataset = RegionDataset(train_df, train_img_dir, train_transform)
val_dataset = RegionDataset(val_df, val_img_dir, val_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=2, pin_memory=True)

In [ ]:
model = models.resnet50(pretrained=True)

# Fine-tune entire model
for param in model.parameters():
    param.requires_grad = True

model.fc = nn.Linear(model.fc.in_features, 15)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

epochs = 15
best_val_acc = 0

print("🚦 Starting training...")
for epoch in range(epochs):
    model.train()
    running_loss = 0
    pbar = tqdm(train_loader, desc=f"📚 Epoch {epoch+1}/{epochs}", leave=False)

    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        pbar.set_postfix({'Loss': f"{running_loss / (pbar.n + 1):.4f}"})

    avg_loss = running_loss / len(train_loader)

    # -------------------------
    # Validation Accuracy
    # -------------------------
    model.eval()
    predictions = []
    ground_truth = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            ground_truth.extend(labels.numpy())

    val_acc = accuracy_score(ground_truth, predictions)
    print(f"✅ Epoch {epoch+1} | Loss: {avg_loss:.4f} | Validation Accuracy: {val_acc:.4f}")

    # Save best model (optional)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), os.path.join(base_path, 'best_model.pth'))

    scheduler.step()


print("🔍 Final evaluation on validation set...")

model.eval()
predictions = []
with torch.no_grad():
    for images, _ in val_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

val_preds_df = pd.DataFrame({
    'id': list(range(369)),
    'Region_ID': [p + 1 for p in predictions]
})

test_df = pd.DataFrame({
    'id': list(range(369, 738)),
    'Region_ID': [1] * 369
})

submission_df = pd.concat([val_preds_df, test_df], ignore_index=True)
submission_df.to_csv(region_output_csv, index=False)
print(f"📁 Submission saved to {region_output_csv}")
print("🎉 All steps completed. Best Validation Accuracy: {:.4f}".format(best_val_acc))


# Angle Prediction

In [ ]:
class AngleDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')
        angle = row['angle']
        if self.transform:
            image = self.transform(image)
        return image, angle

def mean_absolute_angular_error(true, pred):
    true = np.array(true)
    pred = np.array(pred)
    return np.mean(np.minimum(np.abs(true - pred), 360 - np.abs(true - pred)))

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

best_maae = float('inf')

for epoch in range(100):
    model.train()
    running_loss = 0.0
    for images, angles in tqdm(train_loader, desc=f"📚 Epoch {epoch+1}/100"):
        images = images.to(device)
        angles = angles.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, angles)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"✅ Epoch {epoch+1}: Loss = {running_loss / len(train_loader):.4f}")

    model.eval()
    val_preds = []
    val_gts = []
    with torch.no_grad():
        for images, angles in val_loader:
            images = images.to(device)
            angles = angles.numpy()
            outputs = model(images).cpu().numpy().flatten()
            val_preds.extend(outputs)
            val_gts.extend(angles)

    val_preds_clamped = [round(max(0, min(360, p))) for p in val_preds]
    maae_score = mean_absolute_angular_error(val_gts, val_preds_clamped)
    print(f"🎯 Epoch {epoch+1}: Validation MAAE = {maae_score:.2f} degrees")

    if maae_score < best_maae:
        best_maae = maae_score
        val_submission = pd.DataFrame({
            'id': list(range(len(val_preds_clamped))),
            'angle': val_preds_clamped
        })
        dummy_test_submission = pd.DataFrame({
            'id': list(range(len(val_preds_clamped), 738)),
            'angle': [0] * (738 - len(val_preds_clamped))
        })
        final_submission = pd.concat([val_submission, dummy_test_submission], ignore_index=True)
        final_submission.to_csv(lat_long_output_csv, index=False)
        print(f"💾 Best MAAE improved to {best_maae:.2f}, submission saved.")


In [ ]:
anomaly_ids = [95, 145, 146, 158, 159, 160, 161]
val_df = val_df[~val_df.index.isin(anomaly_ids)].reset_index(drop=True)

class GeoDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        target = torch.tensor([row['latitude'], row['longitude']], dtype=torch.float)
        return image, target, row['filename']

# 📦 Transforms
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# 🔄 Dataloaders
train_dataset = GeoDataset(train_df, train_img_dir, transform=train_transform)
val_dataset = GeoDataset(val_df, val_img_dir, transform=val_transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# 🧠 Model & Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
model = model.to(device)

criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-4)

# 📉 Track best model
best_val_mse = float('inf')

# 🏋️‍♂️ Training Loop
epochs = 20
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for images, targets, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # 📊 Validation
    model.eval()
    val_loss = 0
    predictions = []
    with torch.no_grad():
        for images, targets, filenames in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

            # Save predictions for CSV
            for fname, pred in zip(filenames, outputs.cpu().numpy()):
                predictions.append({'filename': fname, 'latitude': pred[0], 'longitude': pred[1]})

    val_mse = val_loss / len(val_loader)
    print(f"✅ Epoch {epoch+1} - Train Loss: {train_loss:.4f} | Val MSE: {val_mse:.4f}")

    # 💾 Save best predictions
    if val_mse < best_val_mse:
        best_val_mse = val_mse
        print(f"🌟 New best MSE: {best_val_mse:.4f}, saving predictions.")
        pd.DataFrame(predictions).to_csv(output_csv, index=False)



# ❌ Remove anomaly IDs from val and preserve original indices
anomaly_ids = [95, 145, 146, 158, 159, 160, 161]
val_df_cleaned = val_df.drop(anomaly_ids).reset_index(drop=False)  # Keep original index as 'index'
val_df_cleaned.rename(columns={'index': 'id'}, inplace=True)       # Rename index to 'id'

# ...
# 📈 Validation
model.eval()
val_preds = []
with torch.no_grad():
    for images, _ in val_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.round().cpu().numpy().astype(int)
        val_preds.extend(preds)

# 📤 Submission file
print("📝 Generating submission...")
val_submission = pd.DataFrame({
    'id': val_df_cleaned['id'].tolist(),  # Use original id
    'Latitude': [lat for lat, lon in val_preds],
    'Longitude': [lon for lat, lon in val_preds]
})

# Add 0,0 for test samples
test_ids = list(range(369, 738))
test_submission = pd.DataFrame({
    'id': test_ids,
    'Latitude': [0] * len(test_ids),
    'Longitude': [0] * len(test_ids)
})

# Combine and save
submission_df = pd.concat([val_submission, test_submission], ignore_index=True)
submission_df = submission_df.sort_values(by='id').reset_index(drop=True)
submission_df.to_csv('2022101005_1.csv', index=False)
print("✅ Submission file '2022101005_1.csv' created!")
